In [1]:
import numpy as np
import pandas as pd

In [3]:
path = 'Iris.csv'

In [13]:
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [21]:
class IrisDataset(Dataset):
    def __init__(self, csv_file_path):
        df = pd.read_csv(csv_file_path)
        df = df.drop('Id', axis=1)
        self.X = df.drop('Species', axis=1).values
        self.y = df['Species'].values
        self._preprocess_data()

    def _preprocess_data(self):
        self.scaler = StandardScaler()
        self.X = self.scaler.fit_transform(self.X)
        self.y = pd.Series(self.y).map({
            'Iris-setosa': 0, 
            'Iris-versicolor': 1, 
            'Iris-virginica': 2
        }).values

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def train_test_split(self, test_size):
        return train_test_split(self.X, self.y, test_size=test_size, random_state=420)

In [22]:
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [23]:
dataset = IrisDataset(path)
X_train, X_test, y_train, y_test = dataset.train_test_split(test_size=0.2)

train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [24]:
model = IrisModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    for data, targets in train_loader:
        data, targets = data.to(device), targets.to(device)

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch+1}/{100}, Loss: {loss.item():.4f}')

Epoch: 10/100, Loss: 0.3042
Epoch: 20/100, Loss: 0.3023
Epoch: 30/100, Loss: 0.0536
Epoch: 40/100, Loss: 0.0591
Epoch: 50/100, Loss: 0.2226
Epoch: 60/100, Loss: 0.0588
Epoch: 70/100, Loss: 0.0152
Epoch: 80/100, Loss: 0.0902
Epoch: 90/100, Loss: 0.0025
Epoch: 100/100, Loss: 0.0106


In [25]:
def test(model, loader, dataset):
    model.eval()
    correct = 0
    total = len(dataset)
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            pred = outputs.argmax(dim=1)
            correct += (pred == targets).sum().item()
    accuracy = (correct / total) * 100
    print(f'Accuracy: {accuracy:.2f}%')

In [26]:
test(model, train_loader, train_dataset)

Accuracy: 99.17%


In [27]:
test(model, test_loader, test_dataset)

Accuracy: 100.00%


In [28]:
new_samples = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Should be Iris-setosa
    [6.0, 3.0, 4.5, 1.5],  # Should be Iris-versicolor  
    [6.5, 3.0, 5.5, 2.0],  # Should be Iris-virginica
    [4.8, 3.0, 1.2, 0.1],  # Another setosa-like
    [7.0, 3.2, 4.7, 1.4],  # Another versicolor-like
    [6.9, 3.1, 5.4, 2.1],  # Another virginica-like
])

In [29]:
def predict(model, scaler, samples):
    scaled_data = scaler.transform(samples)
    tensor_data = torch.FloatTensor(scaled_data).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(tensor_data)
        probabilities = torch.softmax(outputs, dim=1)
        predictions = torch.argmax(outputs, dim=1)

    species = {0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}

    print("=== Prediction Results ===")
    print("Input format: [sepal_length, sepal_width, petal_length, petal_width]")
    print("-" * 70)
    
    for i, (sample, pred, probs) in enumerate(zip(samples, predictions.cpu().numpy(), probabilities.cpu().numpy())):
        print(f"Sample {i+1}: {sample}")
        print(f"  Predicted: {species[pred]}")
        print(f"  Confidence: {probs[pred]:.3f}")
        print(f"  All probabilities:")
        print(f"    Setosa: {probs[0]:.3f}")
        print(f"    Versicolor: {probs[1]:.3f}")
        print(f"    Virginica: {probs[2]:.3f}")
        print("-" * 70)

In [30]:
predict(model, dataset.scaler, new_samples)

=== Prediction Results ===
Input format: [sepal_length, sepal_width, petal_length, petal_width]
----------------------------------------------------------------------
Sample 1: [5.1 3.5 1.4 0.2]
  Predicted: Iris-setosa
  Confidence: 1.000
  All probabilities:
    Setosa: 1.000
    Versicolor: 0.000
    Virginica: 0.000
----------------------------------------------------------------------
Sample 2: [6.  3.  4.5 1.5]
  Predicted: Iris-versicolor
  Confidence: 0.989
  All probabilities:
    Setosa: 0.001
    Versicolor: 0.989
    Virginica: 0.010
----------------------------------------------------------------------
Sample 3: [6.5 3.  5.5 2. ]
  Predicted: Iris-virginica
  Confidence: 0.998
  All probabilities:
    Setosa: 0.000
    Versicolor: 0.002
    Virginica: 0.998
----------------------------------------------------------------------
Sample 4: [4.8 3.  1.2 0.1]
  Predicted: Iris-setosa
  Confidence: 1.000
  All probabilities:
    Setosa: 1.000
    Versicolor: 0.000
    Virginica: